In [11]:
import requests
from bs4 import BeautifulSoup
import csv
import lxml

In [3]:
src = requests.get('https://www.hltv.org/stats/players?matchType=Lan').text

In [9]:
with open('players.html','w',encoding='utf-8') as f:
    f.write(src)

In [30]:
with open('players.html','r',encoding='utf-8') as players_html:
    soup = BeautifulSoup(players_html,'lxml')

players = {}
for player in soup.find_all('td',class_='playerCol'):
    #print(player)
    players[player.text] = 'hltv.org' + player.find('a')['href']
    #players[player.text] = 
    
players




{'ZywOo': 'hltv.org/stats/players/11893/ZywOo?matchType=Lan',
 's1mple': 'hltv.org/stats/players/7998/s1mple?matchType=Lan',
 'ZELIN': 'hltv.org/stats/players/9001/ZELIN?matchType=Lan',
 'zAAz': 'hltv.org/stats/players/1646/zAAz?matchType=Lan',
 'coldzera': 'hltv.org/stats/players/9216/coldzera?matchType=Lan',
 'NiKo': 'hltv.org/stats/players/3741/NiKo?matchType=Lan',
 'device': 'hltv.org/stats/players/7592/device?matchType=Lan',
 'woxic': 'hltv.org/stats/players/8574/woxic?matchType=Lan',
 'XANTARES': 'hltv.org/stats/players/7938/XANTARES?matchType=Lan',
 'vilga': 'hltv.org/stats/players/1572/vilga?matchType=Lan',
 'huNter-': 'hltv.org/stats/players/3972/huNter-?matchType=Lan',
 'kennyS': 'hltv.org/stats/players/7167/kennyS?matchType=Lan',
 'frozen': 'hltv.org/stats/players/9960/frozen?matchType=Lan',
 'Kaze': 'hltv.org/stats/players/8950/Kaze?matchType=Lan',
 'BnTeT': 'hltv.org/stats/players/7131/BnTeT?matchType=Lan',
 'KSCERATO': 'hltv.org/stats/players/15631/KSCERATO?matchType=Lan'

possible features
- ADR
- KDR
- DPR
- IMPACT
- KAST
- RATING
- CONSISTENCY
- k-d diff
- opening kills
- opening duels
- % of maps with 1+ rating
- HS%